In [24]:
#импорт библиотек
import pandas as pd
import numpy as np
import os
import fnmatch
import warnings
import json

In [25]:
#функция, которая преобразовывает таблицу на одном листе в более читаемый вид
def table(df):
    l = len(df.columns)
    df1 = df.iloc[:, :7]
    df_list = []
    for i in range(7, l, 3):
        df_i_0 = df.iloc[0:, i:i+3]
        df_i = pd.concat([df1, df_i_0],axis=1)
        df_i.columns = df_i.iloc[1]
        df_i['Старое наименование причала'] = df_i_0.columns[1]
        df_i['Новое наименование причала'] = df_i_0.iloc[0][1]
        df_i = df_i.rename(columns={df_i.columns[5]: 'Замена судна'})
        df_list.append(df_i.iloc[2:])

    df_con = df_list[0]
    for i in range(1, len(df_list)): 
        df_con = pd.concat([df_con,df_list[i]],axis=0)
    
    return df_con

#функция, которая объединяет таблицы в файле (кроме таблицы РАЗОВЫЕ)
def tables_in_file(file):
    sheets = pd.ExcelFile(file).sheet_names
    df_concat = pd.read_excel(file, sheet_name=sheets[1])
    df_concat = table(df_concat)
    for i in range(2, len(sheets)):
        df = pd.read_excel(file, sheet_name=sheets[i])
        df_concat = pd.concat([df_concat, table(df)], axis=0)
    datef = pd.to_datetime(file[11:21], format='%d.%m.%Y')
    df_concat['Дата подходов'] = datef.strftime('%Y-%m-%d')
    return df_concat

directory = "./"
files = []
files += os.listdir(directory)

file_list = []
for file in files:
    if fnmatch.fnmatch(file, 'Расписание*.xlsx'):
        file_list.append(file)

#объединение таблиц из всех файлов
all_tables = tables_in_file(file_list[0])
for i in range(1, len(file_list)):
    all_tables = pd.concat([all_tables, tables_in_file(file_list[i])], axis=0)
    

In [26]:
#удаление дубликатов
all_tables = all_tables.drop_duplicates()
all_tables

1,Маршрут,Лот,Рейс,Судовладелец,Судно,Замена судна,Комментарий,ШВ,Подход,Отход,Старое наименование причала,Новое наименование причала,Дата подходов
2,Северный,1,2,"ООО ""ТА ""Мегафлот""",Рио,NaN,выходит по расписанию,В,11:27:00,11:32:00,Киевский воказл,Киевский,2023-07-01
3,Северный,1,3,"ООО ""ТА ""Мегафлот""",Рио,NaN,выходит по расписанию,В,14:27:00,14:32:00,Киевский воказл,Киевский,2023-07-01
4,Северный,1,4,"ООО ""ТА ""Мегафлот""",Рио,NaN,выходит по расписанию,В,17:27:00,17:32:00,Киевский воказл,Киевский,2023-07-01
5,Северный,1,5,"ООО ""ТА ""Мегафлот""",Рио,NaN,выходит по расписанию,В,20:27:00,20:32:00,Киевский воказл,Киевский,2023-07-01
6,Северный,2,1,"ООО ""Все Теплоходы""",Радость,Счастье,выходит по расписанию с Китай-город,В,08:42:00,08:47:00,Киевский воказл,Киевский,2023-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,Коломенский,58,7,ИП Зыбин Сергей Александрович,Натали,NaN,не выходит в рейс,В,1900-01-01 00:10:00,1900-01-01 00:15:00,Коломенское .1,Коломенское,2023-07-31
2,Западный,65,1,ИП Бачурин Иван Алексеевич,Чижик-II,NaN,не выходит в рейс,А,10:37:00,10:44:00,Международная выставка,Сити-Экспоцентр,2023-07-31
2,Западный,65,1,ИП Бачурин Иван Алексеевич,Чижик-II,NaN,не выходит в рейс,А,12:24:00,12:29:00,Серебряный Бор 2,Серебряный Бор 2,2023-07-31
2,Западный,65,1,ИП Бачурин Иван Алексеевич,Чижик-II,NaN,не выходит в рейс,А,12:34:00,12:44:00,Троице-Лыково,Троице-Лыково,2023-07-31


In [27]:
#объединение таблиц "РАЗОВЫЕ"
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    all_raz_tables = pd.read_excel(file_list[0], sheet_name='РАЗОВЫЕ')
    for i in range(1, len(file_list)):
        df_raz = pd.read_excel(file_list[i], sheet_name='РАЗОВЫЕ')
        df_raz = df_raz.rename(columns={df_raz.columns[5]: 'Дата подходов '})
        all_raz_tables = pd.concat([all_raz_tables, df_raz], axis=0)

#удаление дубликатов
all_raz_tables = all_raz_tables.drop_duplicates()

In [28]:
#сохранение мастер-таблицы
with pd.ExcelWriter('New_1.xlsx') as writer:
    all_raz_tables.to_excel(writer, sheet_name='РАЗОВОЕ', index=False)
    all_tables.to_excel(writer, sheet_name='РАЙОНЫ', index=False)
    

In [29]:
#сохранение в виде json
json_str1 = all_tables.to_json(orient='records', force_ascii=False)
json_str2 = all_raz_tables.to_json(orient='records', force_ascii=False)

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(json_str1, f, ensure_ascii=False, indent=4)
    json.dump(json_str2, f, ensure_ascii=False, indent=4)

In [31]:
#подсчёт None-значений
sum(all_raz_tables.isnull().sum())+sum(all_tables.isnull().sum())

64920

In [32]:
#подсчёт дубликатов
len(all_raz_tables)-len(all_raz_tables.drop_duplicates())+len(all_tables)-len(all_tables.drop_duplicates())

0